In [1]:
import pandas as pd
import numpy as np
from evaluation_scripts.base import load_experiment, save_for_latex
import os

In [2]:
from evaluation_scripts.experiment_class import Experiment
experiment = Experiment(["20210417", "20220127", "20220412"])
data = experiment.data

In [37]:
## LOAD
strains, _, dictionaries, pathes, df1, _ = load_experiment("20210417")
_, _, _, _, df2, _ = load_experiment("20220127")
_, _, _, _, df3, _ = load_experiment("20220412")
data = pd.concat([df1, df2, df3])
data = data[data.exclude == False]
data.phenotype = data.phenotype.replace({"UI" : "U"})
path = pathes["general_obj"]

In [38]:
def get_clearance(data):
    df = data[ (data.x.isin(["S", "A_r", "B_r"])) & (data.treatment_with != "none") & (data.turnover_strain.isnull() == False)].copy()
    df["cleared"] = df.phenotype == "U"
    cols = ["x", "treatment_with", "cleared"]
    summary = df[cols].groupby(cols[:-1]).sum()
    summary["total"] = df[cols].groupby(cols[:-1]).count()
    summary["f"] = summary.cleared/summary.total
    summary = summary.reset_index()
    clearance = pd.pivot_table(summary, columns = "x", index = "treatment_with", values = "f")
    order = ["A", "B", "AB"]
    clearance = clearance.loc[order,]
    return clearance

clearance = get_clearance(data)

In [39]:
## SAVE
save_for_latex(clearance, [], os.path.join(pathes["tables"], "clearance_mean.csv"))

  treatmentwith    Ar    Br     S
0             A  0.02  0.98  0.97
1             B  0.62  0.01  0.73
2            AB  0.40  0.98  0.86


In [40]:
clearance


x,A_r,B_r,S
treatment_with,,,
A,0.016542,0.975313,0.967422
B,0.621499,0.009390,0.732519
AB,0.398007,0.975062,0.860759
